# Project 4: Web Scraping Indeed.com & Predicting Salaries

In Project 4, we practice two major skills: collecting data via  web scraping and building a binary predictor with Logistic Regression.

We will collect salary information on data science jobs in a variety of markets. Using location, title, and job summary, we'll predict the salary of the job. For job posting sites, this is extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), extrapolating expected salary can help guide negotiations.

Normally, we can use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Q: Why would we want this to be a classification problem?
- A: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Section one focuses on scraping Indeed.com; then we use listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

Scrape job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries. First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract.

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [1]:
URL = 'http://www.indeed.com/'

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup

In [3]:
# read site in soup
r = requests.get(URL)
soup = BeautifulSoup(r.content, "lxml")

# Append to the full set of results
results = soup.findAll('div', { "class" : "result" })


Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some of the more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```

- Make sure these functions are robust and can handle cases where the data/field may not be available
- Test the functions on the results above

In [4]:
# get text
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''
        
# company
def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

# location
def get_location_from_result(result):
    return extract_text(result.find('span', {'class' : 'location'}))

# summary
def get_summary_from_result(result):
    return extract_text(result.find('span', {'class' : 'summary'}))
# title
def get_title_from_result(result):
    return extract_text(result.find('a', {'data-tn-element' : 'jobTitle'}))
# get salary if exists
def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()   
    return None

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results: the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try different city). The second controls where in the results to start and gives 10 results (so we can keep incrementing this by 10 to move further within the list).

#### Complete the following code to collect results from multiple cities and start points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [5]:
# specify city
YOUR_CITY = ['Atlanta, GA', 'Los+Angeles, CA', 'Seattle, WA', 
             'New+York, NY', 'San+Diego, CA', 'San+Francisco, CA', 'Denver CO', 'Boston, MA', 'Austin, TX', ]

In [6]:
# create template URL and max number of results (pages) to pull
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"

# for loop to pull data with bs4
for city in set(YOUR_CITY):
    for start in range(0, 500, 10 ):
        r = requests.get(url_template.format(city, start))
        # Grab the results from the request (as above)
        soup = BeautifulSoup(r.content, "lxml")
        # Append to the full set of results
        results += soup.findAll('div', { "class" : "result" })
   

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
# combine data into dictionaries
rows = []
for result in results:
    if result:
        row = {'company':get_company_from_result(result),
              'location':get_location_from_result(result),
              'summary':get_summary_from_result(result),
              'title':get_title_from_result(result),
               'salary':get_salary_from_result(result)
              }
        rows.append(row)

# create dataframe
import pandas as pd
df = pd.DataFrame(rows)


In [12]:
df.shape

(6701, 5)

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [18]:
# Filter to only the rows that have salary entries
df = df.dropna()
# Remove duplicates
df.drop_duplicates(inplace = True)
# Filter out salary entries referring to week, hour or month
df_2 = df[~(df.salary.astype('str').str.contains('hour'))] # example
df_2 = df[~(df.salary.astype('str').str.contains('week'))]
df_2 = df[~(df.salary.astype('str').str.contains('month'))]


#reset index
df_2 = df.reset_index(drop=True)


In [33]:
df2.head()


,company,location,salary,summary,title
0,All-In Analytics,"Austin, TX","$140,000 - $160,000 a year","Machine Learning Data Scientist. Forecasting, ...",Senior Machine Learning Data Scientist
1,XOR Data Exchange,"Austin, TX","$90,000 a year",XOR is leading a data revolution by giving tra...,Senior Software Engineer/Team Lead
2,Smith Arnold Partners,"Austin, TX","$50,000 - $65,000 a year",Growing international custom research supplier...,Research Analyst
3,Volt Workforce Solutions,"Austin, TX","$120,000 - $150,000 a year",Data mining competition experience preferred (...,Machine Learning Engineer
4,Lighthouse Recruiting,"Austin, TX","$60,000 - $80,000 a year",Join our Medical Laboratory Scientists Groups:...,LCMS Certifying Scientist


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [21]:
import re
import numpy as np
def extract_salary_average(salary_string):
    regex = r'\$([0-9]+,[0-9]+)'
    matches = re.findall(regex, salary_string)
    return np.mean([float(salary.replace(',', '')) for salary in matches ])

In [22]:
# use '.map' to transform salary to new feature
df_2['salary'] = df_2['salary'].map(extract_salary_average)

In [34]:
df_2

,company,location,salary,summary,title,city
0,All-In Analytics,"Austin, TX",150000.0,"Machine Learning Data Scientist. Forecasting, ...",Senior Machine Learning Data Scientist,Austin
1,XOR Data Exchange,"Austin, TX",90000.0,XOR is leading a data revolution by giving tra...,Senior Software Engineer/Team Lead,Austin
2,Smith Arnold Partners,"Austin, TX",57500.0,Growing international custom research supplier...,Research Analyst,Austin
3,Volt Workforce Solutions,"Austin, TX",135000.0,Data mining competition experience preferred (...,Machine Learning Engineer,Austin
4,Lighthouse Recruiting,"Austin, TX",70000.0,Join our Medical Laboratory Scientists Groups:...,LCMS Certifying Scientist,Austin
5,Denver Public Schools,"Denver, CO 80204 (Central West Denver area)",69673.0,Clean data for analysis. Develop data collecti...,"SENIOR RESEARCH ANALYST, TEACHING AND LEARNING...",Denver
6,10Roof Technology,"Denver, CO",105000.0,"Data Scientist, and a UI Developer. Great atti...","Big Data Developer (Spark, Hadoop, Hive)",Denver
7,Mountain Ltd.,"Denver, CO",42500.0,Enter data from legal sources into database. I...,Legal Research Analyst,Denver
8,Lauren Enterprises,"Denver, CO 80226",35000.0,Looking for a Food Scientist:. Secondary and/o...,Food Scientist: Chemistry & Formulations,Denver
9,xentity corporation,"Denver, CO",111000.0,Validate data models (logical and physical) fo...,Government Enterprise Architect,Denver


In [24]:
# save scraped results as a CSV for Tableau/external viz
df_2.to_csv('Df_2Salaries.csv', encoding = 'utf-8')

## Predicting salaries using Logistic Regression

In [ ]:
# load in the the data of scraped salaries


In [36]:
def city_extract(word):
    return word.split(',')[0].split('.')[0].strip()
def state_extract(word):
    return word.split(',')[1].split(' ')[1].strip()

In [37]:
df_2['city'] = pd.DataFrame({'city':df_2['location'].apply(city_extract)})
df_2['state'] = pd.DataFrame({'state':df_2['location'].apply(state_extract)})


In [38]:
df_2

,company,location,salary,summary,title,city,state
0,All-In Analytics,"Austin, TX",150000.0,"Machine Learning Data Scientist. Forecasting, ...",Senior Machine Learning Data Scientist,Austin,TX
1,XOR Data Exchange,"Austin, TX",90000.0,XOR is leading a data revolution by giving tra...,Senior Software Engineer/Team Lead,Austin,TX
2,Smith Arnold Partners,"Austin, TX",57500.0,Growing international custom research supplier...,Research Analyst,Austin,TX
3,Volt Workforce Solutions,"Austin, TX",135000.0,Data mining competition experience preferred (...,Machine Learning Engineer,Austin,TX
4,Lighthouse Recruiting,"Austin, TX",70000.0,Join our Medical Laboratory Scientists Groups:...,LCMS Certifying Scientist,Austin,TX
5,Denver Public Schools,"Denver, CO 80204 (Central West Denver area)",69673.0,Clean data for analysis. Develop data collecti...,"SENIOR RESEARCH ANALYST, TEACHING AND LEARNING...",Denver,CO
6,10Roof Technology,"Denver, CO",105000.0,"Data Scientist, and a UI Developer. Great atti...","Big Data Developer (Spark, Hadoop, Hive)",Denver,CO
7,Mountain Ltd.,"Denver, CO",42500.0,Enter data from legal sources into database. I...,Legal Research Analyst,Denver,CO
8,Lauren Enterprises,"Denver, CO 80226",35000.0,Looking for a Food Scientist:. Secondary and/o...,Food Scientist: Chemistry & Formulations,Denver,CO
9,xentity corporation,"Denver, CO",111000.0,Validate data models (logical and physical) fo...,Government Enterprise Architect,Denver,CO


#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [52]:
# calculate median and create feature with 1 as high salary

median_data = df_2['salary'].median()
median_data
df_2['High_Low'] = df_2['salary'].map(lambda x: 1 if x >= median_data else 0)


### Q: What is the baseline accuracy for this model?

It is 50% if we guess randomly, half the salaries will be below the median and half will be above.

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [55]:
# create statsmodel and summary
import statsmodels.formula.api as sm


In [64]:
model = sm.logit("High_Low ~ state", data=df_2).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.660487
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               High_Low   No. Observations:                  147
Model:                          Logit   Df Residuals:                      139
Method:                           MLE   Df Model:                            7
Date:                Mon, 21 Nov 2016   Pseudo R-squ.:                 0.04556
Time:                        16:25:13   Log-Likelihood:                -97.092
converged:                       True   LL-Null:                       -101.73
                                        LLR p-value:                    0.2339
===============================================================================
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept      -0.1001      0.317     -0.316      0.752        -0.721     0.520
state[T.CO]    -0.5931      0.689     -0.860      0.390        -1.944     0.758
state[T.GA]    -0.7108      0.679     -1.047      0.295        -2.042     0.620
state[T.MA]     0.5521      0.578      0.955      0.339        -0.581     1.685
state[T.NJ]    -0.5931      1.265     -0.469      0.639        -3.072     1.886
state[T.NY]     0.7287      0.443      1.646      0.100        -0.139     1.597
state[T.TX]    -0.3054      0.966     -0.316      0.752        -2.199     1.588
state[T.WA]     0.5055      0.719      0.703      0.482        -0.904     1.915
===============================================================================
"""

The coefficents here are showing a negative relationship(less income) in the states whose coef has a (-) in front and thus a 1 unit increase in the state leads to less of a chance of it being a high paying job.  The states such as NY, WA and MA are associated with a higher probability of finding a more lucrative job. 

One question i have is, why has the CA data been omitted?

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' or 'Manager' is in the title 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [76]:
# create senior, director, and manager dummies
df_2['is_senior'] = df_2['title'].str.contains('Senior','Sr.').astype(int) # example
df_2['is_Manager'] = df_2['title'].str.contains('Manager').astype(int)
df_2['director'] = df_2['title'].str.contains('Director').astype(int)



In [77]:
df_2.director.value_counts()

0    142
1      5
Name: director, dtype: int64

In [78]:
model = sm.logit("High_Low ~ is_Manager + is_senior + is_Manager + state", data=df_2).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.597094
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               High_Low   No. Observations:                  147
Model:                          Logit   Df Residuals:                      137
Method:                           MLE   Df Model:                            9
Date:                Mon, 21 Nov 2016   Pseudo R-squ.:                  0.1372
Time:                        19:35:47   Log-Likelihood:                -87.773
converged:                       True   LL-Null:                       -101.73
                                        LLR p-value:                 0.0009890
===============================================================================
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept      -0.5078      0.359     -1.416      0.157        -1.211     0.195
state[T.CO]    -0.2478      0.709     -0.349      0.727        -1.637     1.142
state[T.GA]    -0.5205      0.729     -0.714      0.475        -1.949     0.908
state[T.MA]     0.7843      0.613      1.280      0.201        -0.417     1.986
state[T.NJ]    -0.1853      1.276     -0.145      0.885        -2.687     2.316
state[T.NY]     0.7262      0.480      1.513      0.130        -0.214     1.667
state[T.TX]    -1.0880      1.220     -0.892      0.373        -3.479     1.303
state[T.WA]     0.8788      0.735      1.195      0.232        -0.562     2.320
is_Manager      0.3585      0.742      0.483      0.629        -1.095     1.812
is_senior       2.6792      0.821      3.263      0.001         1.070     4.288
===============================================================================
"""

Large increase in probability that the job is higher pay if it has senior in title. Its the biggest determining factor out of all.

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [63]:
# scale, (patsy optional), and fit
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from patsy import dmatrix
import patsy
y, X = patsy.dmatrices('High_Low ~ state', data = df_2)
logit = sm.Logit(y,X)
result = logit.fit()

result.summary()

Optimization terminated successfully.
         Current function value: 0.660487
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               High_Low   No. Observations:                  147
Model:                          Logit   Df Residuals:                      139
Method:                           MLE   Df Model:                            7
Date:                Mon, 21 Nov 2016   Pseudo R-squ.:                 0.04556
Time:                        16:24:53   Log-Likelihood:                -97.092
converged:                       True   LL-Null:                       -101.73
                                        LLR p-value:                    0.2339
===============================================================================
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept      -0.1001      0.317     -0.316      0.752        -0.721     0.520
state[T.CO]    -0.5931      0.689     -0.860      0.390        -1.944     0.758
state[T.GA]    -0.7108      0.679     -1.047      0.295        -2.042     0.620
state[T.MA]     0.5521      0.578      0.955      0.339        -0.581     1.685
state[T.NJ]    -0.5931      1.265     -0.469      0.639        -3.072     1.886
state[T.NY]     0.7287      0.443      1.646      0.100        -0.139     1.597
state[T.TX]    -0.3054      0.966     -0.316      0.752        -2.199     1.588
state[T.WA]     0.5055      0.719      0.703      0.482        -0.904     1.915
===============================================================================
"""

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [91]:
dums = pd.get_dummies(df_2['state'], prefix= 'st')

In [94]:
data = df_2.join(dums.ix[:, 'st_CO':])
data.head(5)

,company,location,salary,summary,title,city,state,is_senior,is_Manager,director,High_Low,st_CO,st_GA,st_MA,st_NJ,st_NY,st_TX,st_WA
0,All-In Analytics,"Austin, TX",150000.0,"Machine Learning Data Scientist. Forecasting, ...",Senior Machine Learning Data Scientist,Austin,TX,1,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,XOR Data Exchange,"Austin, TX",90000.0,XOR is leading a data revolution by giving tra...,Senior Software Engineer/Team Lead,Austin,TX,1,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Smith Arnold Partners,"Austin, TX",57500.0,Growing international custom research supplier...,Research Analyst,Austin,TX,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Volt Workforce Solutions,"Austin, TX",135000.0,Data mining competition experience preferred (...,Machine Learning Engineer,Austin,TX,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,Lighthouse Recruiting,"Austin, TX",70000.0,Join our Medical Laboratory Scientists Groups:...,LCMS Certifying Scientist,Austin,TX,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [95]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn import metrics
# features = df_2['state', 'is_senior']
states = []
X = data['st_CO', 'st_GA', 'st_MA', 'st_NJ', 'st_NY', 'st_TX', 'st_WA']
y = data['High_Low']

X_train, X_test, y_train, y_test = train_test_split(X, y)
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print metrics.accuracy_score(y_test, y_pred)

KeyError: ('st_CO', 'st_GA', 'st_MA', 'st_NJ', 'st_NY', 'st_TX', 'st_WA')

In [84]:
from sklearn.cross_validation import cross_val_score
cval_scores = cross_val_score(result, df_2.state, df_2.High_Low, cv=5 )


#for metric in ['accuracy', 'precision', 'recall', 'roc_auc']: # example
    

TypeError: estimator should a be an estimator implementing 'fit' method, <statsmodels.discrete.discrete_model.BinaryResultsWrapper object at 0x162582b90> was passed

### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [ ]:
model = LogisticRegression(penalty = 'l1', C=1.0)

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    

In [ ]:
model.fit(X_scaled, y)

df = pd.DataFrame({'features' : X.design_info.column_names, 'coef': model.coef_[0,:]})
df.sort_values('coef', ascending=False, inplace=True)
df

#### Optional: Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients. Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary. Which entries have the highest predicted salaries?

# Bonus Section: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are most valuable? 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform()

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X_scaled, y, cv=3, scoring=metric)
    print(metric, scores.mean(), scores.std())

In [ ]:
model.fit(X_scaled, y)

df = pd.DataFrame({'features' : vectorizer.get_feature_names(), 'coef': model.coef_[0,:]})
df.sort_values('coef', ascending=False, inplace=True)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# retest L1 and L2 regularization
from sklearn.linear_model import LogisticRegressionCV

model = LogisticRegressionCV()


Score: | /24
------|-------
Identify: Problem Statement and Hypothesis | 
Acquire: Import Data using BeautifulSoup| 
Parse: Clean and Organize Data| 
Model: Perform Logistic Regression| 
Evaluate: Logistic Regression Results	|
Present: Blog Report with Findings and Recommendations		| 
Interactive Tableau visualizations | 
Regularization |
Bonus: Countvectorizer  | 